## Importer les librairies

In [ ]:
using CSV, DataFrames, Distributions, Gadfly, GLM
using Distances
import StatsBase       # Pour la standardisation des variables

## Traiter les données

In [6]:
data = CSV.read("../train.csv")
y_train = data[end]

temp_X_train = data[2:end-1] #omit id and diagnosis
# Estimation des paramètres de la standardisation
tx = StatsBase.fit(StatsBase.ZScoreTransform, Matrix{Float64}(temp_X_train), dims=1)
# Standardisation des variables
X_train = StatsBase.transform(tx, Matrix{Float64}(temp_X_train));

data_test = CSV.read("../test.csv")
X_test = StatsBase.transform(tx, Matrix{Float64}(data_test[2:end]));

id_test = data_test[1]


println(size(X_train))
println(size(X_test))


(455, 10)
(114, 10)


## Faire le modele KNN

In [7]:
function calculateDist(vec1, vec2)
    return euclidean(vec1,vec2)
end

function findIndexesNBiggest(n, tab)
    temp = copy(tab)
    indexes = []
    while (n > 0 && size(tab)[1] > 0)
        _, index_smaller = findmin(temp)
        push!(indexes, index_smaller)
        splice!(temp, index_smaller)
        n -= 1
    end
    return indexes
end

function predict(k, X_train, y_train, X_test) #works best with odd k    
    nb_data = size(X_train)[1]
    
    distances = []
    for elem in 1:nb_data
        push!(distances, calculateDist(X_test, X_train[elem, :]))
    end
    
    indexes_distances = findIndexesNBiggest(k, distances)
    
    nb_0 = 0
    for index in indexes_distances
        if (y_train[index] == 0)
            nb_0 += 1
        end
    end
    
    return convert(Int8, (nb_0 < k - nb_0))
    
end

function knn(k, X_train, y_train, X_test)
    ans = []
    for elem in 1:size(X_test)[1]
        push!(ans, predict(k, X_train, y_train, X_test[elem, :]))
    end
    return ans
end




"KNN.csv"

## K-cross validation

## Trouver le K qui minimise l'erreur selon les métriques

## Rouler le code avec ce K

In [ ]:
k = 3
y_test = knn(k, X_train, y_train, X_test)
prediction = DataFrame(id = id_test, diagnosis = y_test)
CSV.write("KNN.csv",prediction)